In [33]:
from pyspark.sql import SparkSession
import os
import json
from datetime import timedelta,datetime
from pyspark.sql.functions import *
import time

In [34]:
spark = SparkSession \
    .builder \
    .getOrCreate()

In [35]:
base_path = '/home/leavmir/data/json'
override_config_key = 'spark.override.date'
parquet_write_path = '/home/leavmir/data/parquet/'
current_date = datetime.now().strftime("%Y-%m-%d")

In [36]:
try: 
    conf_value = spark.conf.get(override_config_key) 
    paths_to_read = '{}/date={}'.format(base_path,conf_value)
except:
    print("Parameter is not set")
    paths_to_read = '{}/date={}'.format(base_path,current_date)


Parameter is not set


In [48]:
df = spark.read.option("basePath",base_path) \
    .option("multiLine", "True") \
        .json(paths_to_read)

root
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)
 |-- description: string (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- source: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- urlToImage: string (nullable = true)
 |-- date: date (nullable = true)



In [47]:
final_dataframe = df.withColumn("source_name",col("source.name")) \
    .withColumn("timestamp",to_timestamp("publishedAt")) \
    .drop("id","title","urlToImage","source","publishedAt")

+--------------------+--------------------+--------------------+--------------------+----------+---------------+-------------------+
|              author|             content|         description|                 url|      date|    source_name|          timestamp|
+--------------------+--------------------+--------------------+--------------------+----------+---------------+-------------------+
|        Cherlynn Low|Parler, the app t...|Parler, the app t...|https://www.engad...|2022-09-05|       Engadget|2022-09-02 23:19:02|
|        Billy Steele|In 2021, Jabra up...|In 2021, Jabra up...|https://www.engad...|2022-09-05|       Engadget|2022-09-01 20:00:41|
|      Louryn Strampe|Labor Day sales a...|Make the most of ...|https://www.wired...|2022-09-05|          Wired|2022-09-03 15:00:00|
|         WIRED Staff|Collectively, peo...|This week, we exa...|https://www.wired...|2022-09-05|          Wired|2022-09-01 16:00:00|
|         Simon Lucas|Aside from one of...|There's a new pan...|https

In [40]:
# parquet_path = '/home/leavmir/data/parquet/{}' \
#     .format(time.strftime("date=%Y-%m-%d"))
# if not os.path.isdir(parquet_path):
#     os.mkdir(parquet_path)

In [41]:
final_dataframe = df.join(exploded_json_column, "publishedAt") \
    .withColumn("timestamp",to_timestamp("publishedAt")) \
    .drop("id","title","urlToImage","source","publishedAt")

In [42]:
final_dataframe.write.mode("overwrite") \
    .partitionBy("date") \
    .option("partitionOverwriteMode","dynamic") \
    .parquet(parquet_write_path)